# Use-Case 1

In [9]:
import pandas as pd
import os

csv_collection = []
for dirname, _, filenames in os.walk('C:\\Users\skrPlays\Downloads\\tweets'):
    for filename in filenames:
        fullpath= os.path.join(dirname, filename)
        csv_collection.append(fullpath)

In [10]:
df = pd.read_csv(csv_collection.pop(),compression = 'gzip', index_col=0,
                 dtype={'userid':int, 'username':object, 'acctdesc':object, 'location':object,
                        'following':int, 'followers':int, 'totaltweets':int, 'usercreatedts':object,
                        'tweetid':int, 'tweetcreatedts':object, 'retweetcount':int, 'text':object,
                        'hashtags':object, 'language':object, 'coordinates':object, 'favorite_count':int,
                        'extractedts':object})
for data in csv_collection:
    try:
        tmp = pd.read_csv(data, compression = 'gzip', index_col=0,
                          dtype={'userid':int, 'username':object, 'acctdesc':object, 'location':object,
                                 'following':int, 'followers':int, 'totaltweets':int, 'usercreatedts':object,
                                 'tweetid':int, 'tweetcreatedts':object, 'retweetcount':int, 'text':object,
                                 'hashtags':object, 'language':object, 'coordinates':object, 'favorite_count':int,
                                 'extractedts':object})
    except:
        tmp = pd.read_csv(data, index_col = 0)
    df = pd.concat([df, tmp], axis=0)

C:\Users\skrPlays\AppData\Local\Temp\ipykernel_16764\2102135557.py:9: DtypeWarning: Columns (18,21,24,25,28) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp = pd.read_csv(data, compression = 'gzip', index_col=0,


In [11]:
data = df[["userid", "tweetcreatedts", "username", "text", "language"]]
data = data[data["language"]=="en"]
data.isnull().sum()
data_dates =  pd.to_datetime(df['tweetcreatedts'],  errors='coerce', infer_datetime_format=True).dt.date
data_dates.value_counts()

2022-03-07    567745
2022-03-06    566767
2022-03-05    546780
2022-03-08    519385
2022-03-09    493857
               ...  
2022-04-16    332860
2022-04-17    325909
2022-04-27    318809
2022-04-25    317674
2022-02-24    298045
Name: tweetcreatedts, Length: 63, dtype: int64

In [12]:
import nltk
import re
from nltk.corpus import stopwords
import string

nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
stopword=set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["tweet"] = data["text"].apply(clean)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\skrPlays\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
data['tweet_date'] =  pd.to_datetime(data['tweetcreatedts'],  infer_datetime_format=True).dt.date

In [14]:
spam_id_list = []
def findSpamBots():
    freq = data.tweet.value_counts()[0:10]
    bot_tweets = freq.index
    counts = freq.values

    for bt, c in zip(bot_tweets, counts):
        spam_series = data[data.tweet == bt]['userid']
        for ids in spam_series:
            spam_id_list.append(ids)

findSpamBots()
spam_id_list

MemoryError: Unable to allocate 800. MiB for an array with shape (6, 17484308) and data type object

In [ ]:
data['bot_flag']=data['userid'].apply(lambda x: 1 if x in(spam_id_list) else 0)

In [ ]:
(data.groupby('tweet_date')['bot_flag'].value_counts(normalize=True).unstack()).to_csv("bot_en.csv")